In [ ]:
import math 
import numpy as np 
import tensorflow as tf 


In [ ]:
# Thiết lập phương thức get_shape_list nhận đầu vào là 1 tensor 
# có chức năng lấy ra hình dạng của 1 tensor mong muốn 
def shape_list(x):
    """Xử lý hình dạng động trong tensor một cach dõ ràng."""
    # sử dụng hàm shape.as_list để trả về 1 danh sách chứa các giá trị là kích thước 
    # của tensor 
    static = x.shape.as_list()
    # lấy ra kích thước tensor x sử dụng hàm tf.shape 
    dyamic = tf.shape(x)
    # trả về kết quả là danh scahs các chiều của tensor đầu vào theo chỉ số của danh sách static 
    return [dyamic[i] if s is None else s for i , s in enumerate(static)]


# Thiết lập phương thức softmax là một hàm soft max
# nhận đầu vào là tensor x và chỉ số axis =-1
# softmax(x) = [exp(x_i) / Σ_j exp(x_j)]_i
def softmax(x, axis=-1):
    # trừ giá trị của tensor x cho giá trị lớn nhất của tensor này theo chiều cuối 
    # để tránh chàn số khi tính mũ kết quả được lưu vào x 
    # với keepdims = True tức là giữ lại kích thước ban đầu 
    x = x - tf.reduce_max(x , axis=axis, keepdims=True)
    # Thực hiện tính e ^x để các giá trị thực thành giá trị dương kết quả được lưu vào ẽ
    ex = tf.exp(x)

    # trả về kết quả là phép chia giá trị ex cho tổng các phần tử của ex theo chiều axis=-1 
    # để biến các giá trị dương thành các giá trị có tổng bằng 1
    return ex / tf.reduce_sum(ex, axis=axis, keepdims=True)


# Thiết lập phương thức gelu là hàm kích hoạt gelu 
# có chức năng biến đổi đầu vào thành đầu ra cho tensor tham số 
# gelu(x) = x * 0.5 * (1.0 + erf(x / sqrt(2.0)))
def gelu(x):
    # trả về kết quả là phép biến đổi qua công thức trên của tensor x 
    # hàm pow có chức năng xây dựng phép tính mũ lũy thừa ở đây là mũ 3 cho tất cả các giá trị 
    # trong tensor x 
    return 0.5 * x *(1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044175 * tf.pow(x,3))))

# Thiêts lập phương thức norm là hàm chức năng layernormalization 
# được sử dụng để chuẩn hóa lớp , luồng dữ liệu 
# phương thức này nhận đầu vào x là 1 tensor , scope là 1 chuỗi xác định phạm vi 
# biến của tensorflow , params là một từ điển chứa các thông số cấu hình .. 
def norm(x , scope , * , axis=-1, epsilon=1e-5, params=None):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    # tạo một biến phạm vị với tham số phạm vi scope là 1 chuỗi xác định phạm vi 
    with tf.variable_creator_scope(scope):
        # từ tensor đầu vào lấy ra kích thước theo chiều cuối cùng gán cho n_state 
        n_state = x.shape[-1].value
        # kiểm tra xem kiểu dữ liệu của khóa "precision" trong từ điển params có phảo bfloat16
        if params["percision"] == "bfloat16":
            # nếu đúng ta tạo 2 tensor g và b có kích thước = n_state với tham số 
            # khởi tạo = 1 , 0 . Biến g , b là các tham số của phép biến đổi chéo  tuyến tính 
            # có thể được học và điều chỉnh trong quá trình huấn luyện . value 1 , 0 cho phép 
            # giữu nguyên phân bố của tensor x sau khi chuẩn hóa 
            g = tf.get_variable('g', [n_state], initializer=tf.constant_initializer(1, dtype=tf.bfloat16), dtype=tf.bfloat16)
            b = tf.get_variable('b', [n_state], initializer=tf.constant_initializer(0, dtype=tf.bfloat16), dtype=tf.bfloat16)
        # trường hợp còn lại 
        else:
            # ta tạo b và g theo mặc định 
            g = tf.get_variable('g', [n_state], initializer=tf.constant_initializer(1))
            b = tf.get_variable('b', [n_state], initializer=tf.constant_initializer(0))

        # tính trung bình của tensor x theo chiều axis = - 1 gán cho u 
        u = tf.reduce_mean(x , axis=axis, keepdims=True)
        # sau đó tính trung bình tensor x với các giá trị - u ) ^2
        # và gán kết quả cho s
        s = tf.reduce_mean(tf.square(x - u), axis=axis, keepdims=True)

        # nhân tensor x với tensor g rồi sau đó cộng tensor b 
        #  nhân tensor x với biến g và cộng với biến b để thực hiện phép biến đổi tuyến tính chéo
        x = x*g + b
        return x
    

# thiết lập phương thức split_states nhận đầu vào là tensor x 
# và n là một giá trị nguyên mong muốn 
def split_states(x , n):
    """Reshape the last dimension of x into [n, x.shape[-1] / n]"""
    """Định hình lại kích thước cuôi cùng của x nằm trong [n , x.shape[-1] / n]"""
    # sử dụng hàm shape_list để lấy ra kích thước của tensor x dưới dạng list 
    # gán nó cho start và ,m ssuwr dụng dấu sao để nén kết quả của phép shape_list 
    # cho biến start trừ đi chiều cuối cùng được gán cho m 
    *start , m  = shape_list(x)
    # trả về kết quả cuối cùng của x sau khi được biến đổi 
    # nối list start với 1 list = [n , m // n ] thành 1 list mới 
    # các tham số trong list này sẽ là hình dạng mới cho tensor x 
    return tf.reshape(x, start + [n , m//n])

# thiết lập phương thức marge để hợp nhất 2 chiều cuối cùng của tensor thành 1 chiều duy nhất
def merge_states(x):

    """Smash the last two dimensions of x into a single dimension."""
    # lấy ra kích thước của tensor dưới dạng list gán nó cho biến nén *start nếu 
    # x có 4 chiều thì start  sẽ có 2 chiều và 2 chiều cuối gán cho a và b 
    *start, a, b = shape_list(x)
    # trả về 1 tensor mới với shape bằng kết quả phép cộng danh sách start + [a*b]
    # khi đó x sẽ alf 1 tensor với số chiều giảm đi 1
    return tf.reshape(x, start + [a*b])

def dropout(x, pdrop, train):
    if train and pdrop > 0:
        x = tf.nn.dropout(x, rate=pdrop)
    return x

# Thiết lập phương thức Conv1d Là ,một lớp tích chập 
# phương thức này nhận các tham số tensor x , chuỗi ký tự chỉ định phạm vi scope , 
# nf số nguyên  hỉ định số lượng kênh của tín hiệu đầu ra, hay còn gọi là số lượng bộ lọc (filters) của phép tích chập.
def conv1d(x, scope, nf, *, w_init_stdev=0.02, params=None, scale=False):
    # kiểm tra xem có tồn tại khóa "scale_by_depth" và scale có tồn tại 
    if params["scale_by_depth"] and scale: # Scale by sqrt(num_layers), only happens at the final projection before a res block output
        # nhân giá trị độ lệnh phân phối chuẩn với tỷ lệ 1/ tham số khóa layer trong từ điển params (là 1 giá trị nguyên n_layer của model)
         w_init_stdev = w_init_stdev * (1. / math.sqrt(params["n_layer"]))
    # nếu có tồn tại khóa "scale_by_in" = True
    if params["scale_by_in"]: # Scale by sqrt(num_input_features)
        # tương tự như trên ở đây 1 / căn bậc 2 kích thước tensor x theo chiều cuối 
        w_init_stdev = w_init_stdev * (1. / math.sqrt(x.shape[-1].value))

    # khởi tạo một biến phạm vi với chuỗi ký tự chỉ định phạm vi scope 
    with tf.variable_scope(scope):
        # lấy ra danh sách hình dạng tensor x gán chiều cuối cho nx và các chiều còn lại 
        # cho *start 
        *start, nx = shape_list(x)
        # kiểm tra xem giá trị của khóa "precision" trong từ điển params có phải kiểu dữ liệu bfloat16
        if params["precision"] == "bfloat16":
            # nếu có khởi atoj 2 tensor w và b 
            w = tf.get_variable('w', [1, nx, nf], initializer=tf.random_normal_initializer(stddev=w_init_stdev, dtype=tf.bfloat16), dtype=tf.bfloat16)
            b = tf.get_variable('b', [nf], initializer=tf.constant_initializer(0, dtype=tf.bfloat16), dtype=tf.bfloat16)
        
        # trường hợp còn lại khởi tạo w và b theo mặc định 
        else:
            w = tf.get_variable('w', [1, nx, nf], initializer=tf.random_normal_initializer(stddev=w_init_stdev))
            b = tf.get_variable('b', [nf], initializer=tf.constant_initializer(0))
        # sử dụng hàm tf.matmul để thực hiện phép tích chập và sau đó reshape lại hình dạng mong muốn 
        # và gán cho c kết quả là tensor mới có dạng như tensor x nhưng khác chiều cuối cùng 
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

In [ ]:
# Xây dựng mặt nạ attention 
# dựa trên kiến trúc masked self-attention transformer 
# nhận các tham số nd , ns là kích thước 2 chiều của tensor , dtype là kiểu dữ liệu 
def attention_mask(nd, ns, * , dtype):
    # Tạo 1 ma trận i là 1 dãy số  0-> nd -1 i là 1 ma trận mà có chiều thứ 2 = 1
    # với mỗi phần tủ của i trở thành 1 hàng duy nhất 
    i = tf.range(nd)[:None]
    # và tương tự ma trận j 0 -> ns-1 nhưng có 1 chiều duy nhất 
    j = tf.range(ns)

    # so sánh i và j để tạo ma trận boolean m nơi mỗi phần tử m[i, j] là True nếu 
    # điều kiện i >= j - ns + nd ngược lại là False ma trận có dạng như sau 
    # [[True, False, False, False, False],
    # [True, True, False, False, False],
    # [True, True, True, False, False]]
    m = i >= j - ns + nd 
    # sử dụng tf.cast để áp dụng dtype cho ma trận m sau đó trả về kết quả 
    return tf.cast(m, dtype)

# Xây dựng bộ phận self-attention 
# nhận đầu vào tensor đầu vào x ,chuỗi ký tự chỉ định phạm vi các biến trong Tensorflow 
# n_state: Có thể là số lượng trạng thái hoặc đặc trưng mà mô hình sẽ học (ví dụ: kích thước của vector attention).
# past: Đây có thể là một tensor chứa thông tin từ các bước trước đó của mô hình, cho phép mô hình sử dụng thông tin từ quá khứ để đưa ra quyết định hiện tại.
# features = embedding dim 
def attn(x ,scope, n_state, * , past, params, train=False):
    # kiểm tra xem tensor đầu vào x có phải có 3 chiều 
    # nếu không trương trình sẽ bị ngắt 
    assert x.shape.ndims == 3 #Should be [batch_size , sequence_length, features]
    # Sau đó kiểm tra thêm điều kiện 
    assert n_state % params["n_head"] == 0
    # kiểm tra xem tensor chứa thông tin trước đó của mô hình past có tồn tại 
    if past is not None : # nếu như past không = None 
        # kiểm tra điều kiện về hình dạng của tenor này 
        assert past.shape.ndims == 5 # Should be [batch, 2, heads, sequence, features], where 2 is [k, v]

    # thiết lập phương thức split_head để biến đổi tensor đầu vào thành 1 tensor 4 chiều 
    def split_heads(x):
        # features = embedding_dim 
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(split_states(x, params["n_head"]), [0, 2, 1, 3])

    # thiết lập phương thức merge_head có chức năng ngược lại với phương thưcs split_head 
    def merge_heads(x):
        # Reverse of split heads [batch, sequence, embedding_dim]
        return merge_states(tf.transpose(x, [0, 2, 1, 3]))
    
    # Xây dựng phương thức masked_attention 
    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        # lấy ra kích thước của tensor w 
        _, _, nd , ns = shape_list(w)
        # gọi đến phương thức attention_mask để xây dựng mặt nạ attention 
        b = attention_mask(nd, ns, dtype=w.type)
        # định hình lại tensor shape = [1 , 1 , seq_length , seq_length] để phù hợp với tensor 
        # đâu vào 
        b = tf.reshape(b , [1 , 1 , nd , ns])
        # nhân tensor w với b sau đó trừ đi 1 giá trị 1e-9 * 1 -b theo công thức gốc từ bài báo 
        # transformer 
        w = w*b - tf.cast(1e-10,w.type)* (1 - b)
        return w
    
    # xây dựng lớp kiến trúc multihead attention 
    def multihead_attention(q, k ,v):
        # q, k, v have shape [batch, heads, sequence, features(head_dims)]
        # theo công thức tính q * k.T
        # w shape = [batch_size , num_heads , seq_length, num_per_head]
        w = tf.matmul(q, k, transpose_b=True)
        # Nhân w với căn bậc 2 nghịch đảo của tensor v theo kích thước chiều cuối cùng 
        # shape shape = [batch_Size , num_heads , seq_length, seq_length]
        w = w * tf.rsqrt(tf.cast(v.shape[-1].value, w.type))

        # áp dụng mặt nạ attention lên tensor w 
        w = mask_attn_weights(w)
        # áp dụng hàm softmax cho tensor này 
        w = softmax(w)

        # thêm attention_dropout 
        w = dropout(w, params["attn_dropout"], train)

        # tính toán attn score * v
        a = tf.matmul(w, v)
        # return a
        return a 
    
    # xây dựng một biến phạm vi được chỉ định bởi chuỗi ký tự scope 
    with tf.variable_creator_scope(scope):
        # sử dụng phép tham chiếu văn bản với lớp tích chập conv1d
        c = conv1d(x, 'c_attn', n_state*3 , params=params)
        # áp dụng hàm spilit_head cho toàn bộ q, k ,v 
        # [batch_size , seq_length,3, embedding_dim]
        q , k , v = map(split_heads, tf.split(c, 3 , axis=2))
        # sử dụng hàm stack để tách 2 tensor k , v theo chiều thứ 2 
        present = tf.stack([k, v], axis=1) 
        # kiểm tra xem trạng thái biểu diễn trước đó pats cps tồn tại 0 
        if past is not None: 
            # tách qp và qv từ past
            pk, pv = tf.unstack(past, axis=1)
            # nối chúng với vector k , v tương ứng 
            k = tf.concat([pk, k], axis=-2)
            v = tf.concat([pv, v], axis=-2)

        # áp dụng phép multihead_attention cho q , k , v
        a = multihead_attention(q, k ,v)
        # chuyển vị kích thước của a giảm kích thước chiểu cuối cùng 
        # TĂNG chiều [-2] lên 
        a = merge_heads(a)
        # áp dụng phép tham chiếu văn bản 
        a = conv1d(a, 'c_proj', n_state, params=params)
        a = dropout(a, params["res_dropout"], train)
        # a shape = [batch_size, seq_length, hiddent_sizr]
        return a, present


In [ ]:
#thiết lập lớp đa xử lý trong kiến trúc Transformer original 
def mlp(x, scope, n_state, *, params, train=False):
    # khởi tạo một biến phạm vi được chỉ định bởi 1 chuỗi ký tự
    with tf.variable_scope(scope):
        # lấy ra hình kích thước chiều cuối cùng của tensor đầu vào
        nx = x.shape[-1].value
        # áp dụng hàm kích hoạt gelu lên đầu ra của lớp tham chiếu conv1d cho tensor x
        h = gelu(conv1d(x, 'c_fc', n_state, params=params))
        # áp dụng thêm 1 lớp conv1d 
        h2 = conv1d(h, 'c_proj', nx, params=params, scale=True)
        #  dropout_mlp 
        h2 = dropout(h2, params["res_dropout"], train)
        return h2

# Xây dựng khối xử lý toàn phần tương tự như MLP 
def block(x, scope, *, past, params, train=False):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        # áp dụng lớp norm cho đầu ra lớp chú ý 
        a, present = attn(norm(x, 'ln_1', params=params), 'attn', nx, past=past, params=params, train=train)
        x = x + a
        # áp dụng lớp mlp cho kết quả của đầu ra normlization layer
        m = mlp(norm(x, 'ln_2', params=params), 'mlp', nx*4, params=params, train=train)
        x = x + m
        # cộng đầu vào với đầu ra 
        return x, present

# thiết lập phương thức trả về 1 danh sách các giá trị batch_size , num_layer , 2 , num_head , seq_length, embedding_dim , num_head
# là các giá trị tham số trong từ điển params 
def past_shape(*, params, batch_size=None, sequence=None):
    return [batch_size, params["n_layer"], 2, params["n_head"], sequence, params["n_embd"] // params["n_head"]]


# Xây dựng phương thức tiện ích dùng để thêm 1 trục mới vào tensor nguồn
def expand_tile(value, size):
    """Thêm 1 trục mới vào kích thước cố định."""
    # chuyển đổi mảng value thành 1 tensor 
    value = tf.convert_to_tensor(value)
    # lấy ra số lượng chiêu của tensor 
    ndims = value.shape.ndims 
    # Lặp lại tensor đã được mở rộng chiều (từ bước 3) theo mẫu được chỉ định trong danh sách (từ bước 4),
    # tạo ra một tensor mới với trục mới được thêm vào có kích thước là size.
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)


# thiết lạp phương thức tạo tensor vị chí cho mỗi token 
def positions_for(tokens , past_length):
    # Lấy kích thước batch từ tensor tokens, tức là số lượng chuỗi trong batch.
    batch_size = tf.shape(tokens)[0]
    # Lấy số bước (số lượng token) từ tensor tokens, tức là độ dài của mỗi chuỗi.
    nsteps = tf.shape(tokens)[1]
    # Tạo một tensor vị trí cho mỗi bước, bắt đầu từ past_length. 
    # past_length có thể là độ dài của các token đã được xử lý trước đó 
    # (trong trường hợp của mô hình autoregressive như GPT-2),
    # và tf.range(nsteps) tạo ra một dãy số từ 0 đến nsteps - 1.
    return expand_tile(past_length + tf.range(nsteps), batch_size)
    # Kết quả là một tensor mới, trong đó mỗi chuỗi trong batch có một dãy vị trí tương ứng, bắt đầu từ past_length và tăng dần theo mỗi bước


def _assert_float_dtype(dtype):
    if not dtype.is_floating:
        raise ValueError("Expected floating point type, got %s." % dtype)
    return dtype

In [ ]:
def model(X, params, labels=None, past=None, scope='model', reuse=False, train=False):
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        if params["precision"] == "bfloat16":
            wpe = tf.get_variable('wpe', [params["n_ctx"], params["n_embd"]], # Position encoding
                             initializer=tf.random_normal_initializer(stddev=0.01, dtype=tf.bfloat16), dtype=tf.bfloat16)
            wte = tf.get_variable('wte', [params["n_vocab"], params["n_embd"]], # Text encoding
                             initializer=tf.random_normal_initializer(stddev=0.02, dtype=tf.bfloat16), dtype=tf.bfloat16)

        else:
            wpe = tf.get_variable('wpe', [params["n_ctx"], params["n_embd"]], # Position encoding
                                initializer=tf.random_normal_initializer(stddev=0.01))
            wte = tf.get_variable('wte', [params["n_vocab"], params["n_embd"]], # Text encoding
                                initializer=tf.random_normal_initializer(stddev=0.02))
        past_length = 0 if past is None else tf.shape(past)[-2]

        wpe = dropout(wpe, params["embed_dropout"], train)
        wte = dropout(wte, params["embed_dropout"], train)

        h = tf.gather(wte, X) + tf.gather(wpe, positions_for(X, past_length))

        # Transformer
        presents = []
        pasts = tf.unstack(past, axis=1) if past is not None else [None] * params["n_layer"]
        assert len(pasts) == params["n_layer"]
        for layer, past in enumerate(pasts):
            h, present = block(h, 'h%d' % layer, past=past, params=params, train=train)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f', params=params)

        h_flat = tf.reshape(h, [batch*sequence, params["n_embd"]])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, params["n_vocab"]])
        results['logits'] = logits
        return results